# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# Load the csv into a data frame
weather_data = pd.read_csv("output/world_cities.csv")
# Check it
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vendas Novas,38.6771,-8.4579,80.71,38,0,5.66,PT,1631800657
1,Arlit,18.7369,7.3853,105.08,9,5,3.29,NE,1631800658
2,Santa Cruz de la Sierra,-17.8000,-63.1667,69.26,52,75,12.66,BO,1631800406
3,Montepuez,-13.1256,38.9997,92.12,23,5,5.82,MZ,1631800661
4,Cockburn Town,21.4612,-71.1419,84.16,74,40,11.50,TC,1631800662


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [22]:
# Step 1: Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = True, max_intensity = 100,
                                 point_radius = 20)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Step 1 - Find only cities that meet the following ideal weather conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_weather_data = weather_data.loc[(weather_data["Max Temp"] < 80) & 
                                        (weather_data["Max Temp"] > 70) &
                                        (weather_data["Wind Speed"] < 10) & 
                                        (weather_data["Cloudiness"] == 0)]

# Step 2 - Drop any rows with null values
ideal_weather_data = ideal_weather_data.dropna()

# Step 3 - Select only the top 10 cities from this data
ideal_weather_data_top_10 = ideal_weather_data.head(10)
ideal_weather_data_top_10 # There are 10 rows

# Step 4 - export the data to a csv (to avoid a warning message later)
ideal_weather_data_top_10.to_csv("output/ideal_weather_data_top_10.csv",index = False)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# Store the ideal weather data into a variable named hotel_df
hotel_df = pd.read_csv("output/ideal_weather_data_top_10.csv")

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Set the parameters of the search
    params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": f"{hotel_df.loc[index, 'Lat']}, {hotel_df.loc[index, 'Lng']}"
        }
    
    # assemble url and make API request
    try: 
        response = requests.get(base_url, params=params).json()
        # Get the hotel name
        hotel_name = response["results"][0]["name"]
        # Assign it to the hotel DataFrame
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        # Log the result
        print(f"The nearest hotel to the city of {hotel_df.loc[index, 'City']} is {hotel_name}.")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("-----------------------------------------------------")


The nearest hotel to the city of Jacareacanga is HOTEL VIANA.
-----------------------------------------------------
The nearest hotel to the city of Siloam Springs is Holiday Inn Express & Suites Siloam Springs, an IHG Hotel.
-----------------------------------------------------
The nearest hotel to the city of Saint George is Red Lion Hotel & Conference Center St. George.
-----------------------------------------------------
The nearest hotel to the city of Santa Fe is InterTower Hotel.
-----------------------------------------------------
The nearest hotel to the city of Kherson is Greenstone.
-----------------------------------------------------
The nearest hotel to the city of Vila is Hotel Castrum Villae.
-----------------------------------------------------
The nearest hotel to the city of Adelaide is InterContinental Adelaide, an IHG Hotel.
-----------------------------------------------------
The nearest hotel to the city of Hurricane is My Place Hotel- Hurricane, UT.
---------

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info
city_locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add the hotel markers to the heat map
markers = gmaps.marker_layer(city_locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display the improved heat map
fig

Figure(layout=FigureLayout(height='420px'))